In [1]:
from PIL import Image
from IPython.display import display

import PyNvCodec as nvc
import numpy as np

In [2]:
url = "../tests/data/test.mp4"

In [3]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [ ]:
# GPU-accelerated decoder
pyDec = nvc.PyDecoder(
    url,
    {},
    gpu_id=0)

# GPU-accelerated converter
pyCvt = nvc.PySurfaceConverter(
    pyDec.Format(),
    nvc.PixelFormat.RGB,
    gpu_id=0)

# GPU-accelerated Surface downloader
pyDwn = nvc.PySurfaceDownloader(gpu_id=0)

In [ ]:
# Raw decoded Surface
surf_src = nvc.Surface.Make(
    format=pyDec.Format(),
    width=pyDec.Width(),
    height=pyDec.Height(),
    gpu_id=0)

# Raw Surface, converted to RGB
surf_dst = nvc.Surface.Make(
    format=nvc.PixelFormat.RGB,
    width=pyDec.Width(),
    height=pyDec.Height(),
    gpu_id=0)

# Numpy array which contains decoded RGB Surface
frame = np.ndarray(
    dtype=np.uint8,
    shape=surf_dst.HostSize())

# Packet data
pkt_data = nvc.PacketData()

while True:
    # Decode single Surface
    success, details = pyDec.DecodeSingleSurface(surf_src, pkt_data)
    if not success:
        print(details)
        raise StopExecution

    # Convert tot RGB
    success, details = pyCvt.Run(surf_src, surf_dst)
    if not success:
        print(details)
        raise StopExecution

    # Copy pixels to numpy ndarray
    if not pyDwn.Run(surf_dst, frame):
        print("Failed to download surface")
        raise StopExecution

    # Reshape to proper dimensions.
    # Some video formats like NV12 require to reshape to 'raw' dimensions.
    # Others like RGB24 require to reshape to 'final' dimensions. Hence this
    # feature isn't yet supported by VALI out of the box. Have to do that by
    # hand. Sic !
    res_frame = np.reshape(
        frame,
        (pyDec.Height(), pyDec.Width(), 3))

    # Display in notebook
    display(Image.fromarray(res_frame))

    # Output packet data
    print(pkt_data)